<a href="https://colab.research.google.com/github/Sebastian009/03MAIR---Algoritmos-de-Optimizacion---2023/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos(V2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Juan Sebastian Casallas Estrella  <br>
Url: https://github.com/Sebastian009/03MAIR---Algoritmos-de-Optimizacion---2023<br>
Google Colab: https://colab.research.google.com/drive/xxxxxxxxxxxxxxxx <br>
Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Configuración de Tribunales

Descripción del problema:(copiar enunciado)

....







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

In [ ]:
#Respuesta


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

In [ ]:
#Respuesta


#Diseño
- ¿Que técnica utilizo? ¿Por qué?

In [ ]:
#Respuesta